In [2]:
import datasets
from datasets import load_dataset

from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

In [1]:
MODEL_NAME = "kykim/albert-kor-base"

dataset = load_dataset('custom_squad_v2', split='train')
squad = dataset.train_test_split(0.1)

squad.to_json('train_modified.json')
# squad['validation'] = squad['test']
# del squad['test']

NameError: name 'load_dataset' is not defined

In [87]:
print("Context: ", squad["train"][0]["context"])
print("Question: ", squad["train"][0]["question"])
print("Answer: ", squad["train"][0]["answers"])

Context:  SK텔레콤(대표이사 사장 박정호)은 20일부터 갤럭시S20 예약 판매에 돌입하고 오는 27일부터 개통을 시작한다고 밝혔다. 갤럭시S20 플러스 아우라 블루 컬러는 전 세계에서 오직 SK텔레콤에서만 구매 가능하다. 블루 컬러는 기존에 갤럭시 단말을 구매한 고객들로부터 큰 인기를 끌었다. SK텔레콤은 갤럭시S20 예약 판매를 맞아 온∙오프라인 채널에서 대대적인 ‘블루 마케팅’을 펼친다. 먼저 SK텔레콤 공식 온라인샵 T월드 다이렉트에서 갤럭시S20을 예약 및 구매한 고객 가운데 선착순 3천명에게 ‘아우라 블루 패키지’를 제공한다. ‘아우라 블루 패키지’는 갤럭시S20 단말과 아우라 블루 에코백, 신세계 상품권 3만원으로 구성돼 있다. T월드 다이렉트에서 예약한 모든 고객에게는 ▲PANTONE 액세서리 5종 ▲블루투스 이어폰 ▲맨프로토 미니 삼각대 ▲고속충전 지원 액세서리 총 8종의 사은품 가운데 1종을 증정한다. 또 예약 후 개통한 고객 가운데 100명을 추첨해 삼성 에어드레서, 무풍큐브, 블루스카이 공기청정기 등을 제공한다. 매장 방문이 어려운 고객은 T월드 다이렉트에서 ‘오늘도착’ 서비스를 신청하면, 원하는 시간과 장소에서 갤럭시S20을 손쉽게 받아 볼 수 있다. 전문 상담사가 직접 고객을 찾아가 개통부터 데이터 이전까지 마무리해 준다. 갤럭시S20을 저렴하게 구매하고 싶은 고객은 ▲제휴카드 더블할인 ▲T안심보상 ▲T모아쿠폰 등 다양한 구매 혜택을 활용해 최대 130만원까지 할인 받을 수 있다. 추후에도 새 갤럭시 단말로 기기변경을 원하는 고객은 ‘5GX 클럽 갤럭시S20’ 가입을 고려해 볼 만 하다. 월 이용료 7,480원(부가세 포함)을 내면 24개월 후 새 갤럭시 단말로 기기변경 시 출고가의 50% 수준까지 할인 받는다. SK텔레콤은 3월 멤버십 혜택으로 T Day에 ‘5GX 찬스’를 신설해 5GX 요금제 이용 고객을 대상으로 추가 혜택을 제공한다. 11번가 갤럭시S20 액세서리 22% 할인, 도미노피자 60% 할인 등 갤럭시S20 구매

In [88]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

loading configuration file config.json from cache at /home/robust/.cache/huggingface/hub/models--monologg--koelectra-base-v3-discriminator/snapshots/68b30cd259f34a4b5aa8786392612ba2a2617fcc/config.json
Model config ElectraConfig {
  "_name_or_path": "monologg/koelectra-base-v3-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 350

In [89]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [90]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

In [91]:
data_collator = DefaultDataCollator()
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

loading configuration file config.json from cache at /home/robust/.cache/huggingface/hub/models--monologg--koelectra-base-v3-discriminator/snapshots/68b30cd259f34a4b5aa8786392612ba2a2617fcc/config.json
Model config ElectraConfig {
  "_name_or_path": "monologg/koelectra-base-v3-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 350

In [92]:
BATCH_SIZE = 256

training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False, # True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [93]:
trainer = Trainer(
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_4357/364849530.py:1 in <module>                                            │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_4357/364849530.py'                   │
│ /home/robust/miniconda3/lib/python3.9/site-packages/transformers/trainer.py:314 in        │
│ __init__                                                                                  │
│                                                                                           │
│    311 │   │   │   args = TrainingArguments(output_dir=output_dir)                        │
│    312 │   │   self.args = args                                                           │
│    313 │   │   # Seed must be set before instantiating the model when using model         │
│ ❱  314 │   │   enable_full_determinism(self.args.seed) if self.args.full_determinism else │
│    315 │   │   self.hp_name = None                                                        │
│    316 │   │   self.deepspeed = None                                                      │
│    317 │   │   self.is_in_train = False                                                   │
│                                                                                           │
│ /home/robust/miniconda3/lib/python3.9/site-packages/transformers/trainer_utils.py:93 in   │
│ set_seed                                                                                  │
│                                                                                           │
│    90 │   random.seed(seed)                                                               │
│    91 │   np.random.seed(seed)                                                            │
│    92 │   if is_torch_available():                                                        │
│ ❱  93 │   │   torch.manual_seed(seed)                                                     │
│    94 │   │   torch.cuda.manual_seed_all(seed)                                            │
│    95 │   │   # ^^ safe to call this function even if cuda is not available               │
│    96 │   if is_tf_available():                                                           │
│                                                                                           │
│ /home/robust/miniconda3/lib/python3.9/site-packages/torch/random.py:40 in manual_seed     │
│                                                                                           │
│    37 │   import torch.cuda                                                               │
│    38 │                                                                                   │
│    39 │   if not torch.cuda._is_in_bad_fork():                                            │
│ ❱  40 │   │   torch.cuda.manual_seed_all(seed)                                            │
│    41 │                                                                                   │
│    42 │   return default_generator.manual_seed(seed)                                      │
│    43                                                                                     │
│                                                                                           │
│ /home/robust/miniconda3/lib/python3.9/site-packages/torch/cuda/random.py:113 in           │
│ manual_seed_all                                                                           │
│                                                                                           │
│   110 │   │   │   default_generator = torch.cuda.default_generators[i]                    │
│   111 │   │   │   default_generator.manual_seed(seed)                                     │
│   112 │                                           